# **Extraindo Características LBP dos patches**

- **Conectando com Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **1 - Carrega bibliotecas**

In [ ]:
from skimage import io, color                                                    #Módulo para manipulação de imagens
from os import listdir
from os.path import isfile, join
import random
from skimage.feature import local_binary_pattern                                 #Módulo para cálculo do LBP
from numpy import arange
import numpy as np                                                               #Módulo para a manipulação de arrays
import pandas as pd                                                              #Módulo para a manipulação de dataframes
import matplotlib.pyplot as plt                                                  #Módulo para a manipulação de gráficos
import os, glob                                                                  #Módulos para manipular estruturas de diretório
import cv2                                                                       #Módulo para manipulação de imagens

# **2- Aplicando LBP em Patches**

In [ ]:
PATHDST = './Output_patches' # Diretório para salvar os resultados

In [ ]:
#-----------------------------------------------------------------------------#
# Função para extração de características com algoritmo LBP
#-----------------------------------------------------------------------------#
def feature_extraction_LBP(patches, labels, P, R, METHOD):
  features = []
  labels_all = []

  for i, patch in enumerate(patches):
    # Conversão para escala de cinza
    grayscale_image = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    lbp = local_binary_pattern(grayscale_image, P, R, METHOD)
    n_bins = int((lbp.max() + 1))
    hist, bins = np.histogram(lbp.flatten(), bins=n_bins, range=[0, n_bins], density=False)
    features.append(hist)

  for label in (labels):
    filename_parts = label.split('_')
    patch = filename_parts[0]
    sp = filename_parts[2]
    amostra = filename_parts[4]
    code = filename_parts[6].split('.')[0]
    dev = filename_parts[5]
    label = (patch, sp, amostra, code, dev)
    labels_all.append(label)

  df1 = pd.DataFrame(features, columns=[f"V{str(col)}_{METHOD[0:3]}{P}{R}" for col in range(len(features[0]))])
  df2 = pd.DataFrame(labels_all, columns=['Patch', 'Class', 'Sample', 'Id', 'Dev'])
  df2['Class'] = df2['Class'].astype(int)

  # Substituição dos nomes das espécies
  data = pd.read_excel('./Class.xlsx')
  species_mapping = data.set_index('n')['Specie'].to_dict()
  df2['Class'] = df2['Class'].map(species_mapping)

  df = pd.concat([df1, df2], axis=1)

  path = PATHDST + '/Features'

  if not os.path.exists(path):
    os.makedirs(path)
    print("Folder %s created!" % path)
  else:
    print("Folder %s already exists" % path)

  df.to_csv(f'{path}/LBP_{METHOD}_{P}{R}_Patches.csv', index=False, header=True, sep=',', encoding='utf-8')

  return df

In [ ]:
#-----------------------------------------------------------------------------#
# Carregando array com patches
#-----------------------------------------------------------------------------#

# Load npz
data_train = np.load('./Output_patches/patches_and_labels_train.npz') # Dados de treino
data_test = np.load('./Output_patches/patches_and_labels_test.npz') # Dados de teste

# # Acessando os dados
patches_train = data_train['patches']
labels_train = data_train['labels']

patches_test = data_test['patches']
labels_test = data_test['labels']


### ***2.1 - $LBP^{riu2}_{P,R}$ - LBP UNIFORME E INVARIANTE À ROTAÇÃO***

In [ ]:
# Extraindo características do LBP Uniforme (3 combinações)
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=8, R=1, METHOD='uniform')   # 10 features
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=16, R=2, METHOD='uniform')  # 18 features
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=24, R=3, METHOD='uniform')  # 26 features

# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=8, R=1, METHOD='uniform')   # 10 features
# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=16, R=2, METHOD='uniform')  # 18 features
# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=24, R=3, METHOD='uniform')  # 26 features

### ***2.2 - $LBP^{u2}_{P,R}$ - LBP UNIFORME E NÃO INVARIANTE À ROTAÇÃO***

In [ ]:
# Extraindo características do LBP Uniforme e Não Invariante (3 combinações)
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=8, R=1, METHOD='nri_uniform')   # 59 features
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=16, R=2, METHOD='nri_uniform')  # 243 features
feature_extraction_LBP(patches = patches_test, labels = labels_test, P=24, R=3, METHOD='nri_uniform')  # 555 features

# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=8, R=1, METHOD='nri_uniform')   # 59 features
# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=16, R=2, METHOD='nri_uniform')  # 243 features
# feature_extraction_LBP(patches = patches_train, labels = labels_train, P=24, R=3, METHOD='nri_uniform')  # 555 features